In [1]:
!pip install -qq sentence_transformers pinecone-client openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [61]:
!pip install --upgrade --quiet  docx2txt langchain_community langchain langchain-pinecone langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:00


In [66]:
import os
from pinecone import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from google.colab import userdata
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

In [89]:
#This is for embedding. In here, one LM model from huggingface used.

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


In [4]:
# #Function to split long documents in to smaller parts
# def split_text_into_chunks(plain_text, max_chars=2000):
#     text_chunks = []
#     current_chunk = ""
#     for line in plain_text.split("\n"):
#         if len(current_chunk) + len(line) + 1 <= max_chars:
#             current_chunk += line + " "
#         else:
#             text_chunks.append(current_chunk.strip())
#             current_chunk = line + " "
#     if current_chunk:
#         text_chunks.append(current_chunk.strip())
#     return text_chunks

In [ ]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [59]:
loader = Docx2txtLoader("DataLaw.docx")

In [68]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
index_name = "lawdata"
docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [84]:
#This function is responsible for matching the input string with alread existing data on vector database.

def find_match(query,k):

    result = docsearch.similarity_search(query)
    top_k_results = result[:k]  # Get top k results

    # Check if results exist (avoid potential errors)
    if not top_k_results:
        return "No relevant documents found."

    # Efficiently merge content using list comprehension
    merged_content = "\n\n".join([d.page_content for d in top_k_results])

    return merged_content
find_match("Hello how are you", 2)

'Legal entities or natural persons that are the owners, lessees or other users of agricultural land (hereinafter: owner, lessee or other user of agricultural land) must allow other persons to carry out, in accordance with regulations, beekeeping, hunting and the recreational gathering of fruits of wild plants, herbaceous wild plants, mushrooms and wild animals on non-arable agricultural land owned or leased by them or otherwise allotted to them and allow other persons free access to non-arable agricultural land provided that no damage is caused by doing so.\n\nPersons responsible for damage to land or crops shall be liable for compensation for damage to the owner, lessee or other user of agricultural land or crops in accordance with regulations.\n\nArticle 6\n\n-         optimally absorbs, retains and releases water. The Government of the Republic of Slovenia (hereinafter: the Government) shall prescribe detailed criteria for permanent soil fertility and plant pollution and mandatory m

In [79]:
from openai import OpenAI

client = OpenAI()

def create_prompt(context,query):
    #Todo: Should be generated with the context/contexts we find by doing semantaic search
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages = [
    {
        "role": "system",
        "content": "You are a prompt engineer. You take in a context and query. Context is result from the database, while query is the user question. Create a prompt for a LLM model based on both. Context: " + context + " Query: " + query
    }
],
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    )
    return response.choices[0].message

In [80]:
def generate_answer(prompt, context):
    #Todo: Pass the generated prompt and pass it to gpt-3 to get answers.
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-16k",
      messages = [
      {
          "role": "system",
          "content": "Answer this question correctly based on the context. Context: " + context
      }
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      )
    return response.choices[0].message

In [87]:
def user_query(query):
  #Todo: Make all the things together.
  # Search the database
  context = find_match(query,1)
  # generate a prompt
  prompt = create_prompt(context,query)
  # getting the answer:
  answer = generate_answer(prompt, context)
  # returning the answer
  return answer.content

In [88]:
user_query("How can I do this?")

'If an owner, lessee, or other user of agricultural land does not comply with the requirements stated in paragraph one of this Article, the agricultural inspection authority will impose appropriate measures through a decision. The owner, lessee, or other user must complete these measures within one year, as specified by the decision.'

## Review

This is a RAG system.
I'm sorry i couldn't use the functions, and that's because I have been using langchain for almost a year and I feel it's way easier to load the data that way.

I also used langchain_pinecone library which makes it easy to create the vector database rather than going another way round.

My recommendations:

- We can make this process more efficient by using langchain modules
- Integrating langchain agents will also make the process much more efficient.
- The create prompt method can be reviewed to make sure that we are not using too much of credits when getting information from the LLM.
- We can also finetune other llms like mistral to make sure we are getting a better response.